Install
memory_profiler **package**

In [2]:
!pip install memory_profiler

Input cell - for create initial node

In [33]:

# %%time
%load_ext memory_profiler
import queue
import math
import random


class Node:
    def __init__(self, n, i, j, state, costPath, hCost, goalPath):
        self.n = n
        self.i = i
        self.j = j
        self.state = state
        self.costPath = costPath
        self.hCost = hCost
        self.goalPath = goalPath

    def __lt__(self, other):
        return self.costPath + self.hCost > other.costPath + other.hCost

# --------------------------------------------------------------------------------

def printState(state):
    n = int(math.sqrt(len(state)))
    for i in range(n*n):
        if i % n == 0 and i != 0:
            print()
        print(state[i], end=' ')
    print()

def hash(node):
    stateStr = ''
    for num in node.state:
        stateStr += str(num) + " "
    return stateStr

def getBlank(state):
    n = int(math.sqrt(len(state)))
    for i in range(n*n):
      if (state[i] == 0):
        return i // n, i % n

# -----------------------------------------------------------

def flatH(state):
    l = int(math.sqrt(len(state)))
    flatH = []
    for i in range(l):
        for j in range(l):
            flatH.append(state[l*j + i])
    return flatH

def flatV(state):
    return state

def inversionDistance(state):
    n = len(state)
    invCount = 0
    for i in range(n):
        if state[i] == 0:
            continue
        for j in range(i+1, n):
            if state[i] > state[j] and state[j] != 0:
                invCount+=1
    return invCount


def idHeuristic(state):
    # width = round(sqrt(number of tiles + 1))
    n = int(math.sqrt(len(state) - 1))
    invCountV = inversionDistance(flatV(state))
    invCountH = inversionDistance(flatH(state))
    v = invCountV // n + invCountV % n
    h = invCountH // n + invCountH % n
    return v + h

# --------------------------------------------------------------------------------

def getSuccessors(node):
    successors = []
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    for dir in directions:
        new_i = node.i + dir[0]
        new_j = node.j + dir[1]
        if 0 <= new_i < node.n and 0 <= new_j < node.n:
            new_state = node.state.copy()
            newHCost = idHeuristic(new_state)
            new_state[node.n * node.i + node.j], new_state[node.n * new_i + new_j] = new_state[node.n * new_i + new_j], new_state[node.n * node.i + node.j]
            successors.append(Node(node.n, new_i, new_j, new_state, node.costPath + 1, newHCost, node.goalPath + str(new_state[node.n * node.i + node.j]) + " "))
    return successors

def ucs(initialNode, goalState):
    frontier = queue.PriorityQueue()
    reached = {}

    frontier.put((initialNode.costPath, initialNode))
    reached[hash(initialNode)] = initialNode

    while not frontier.empty():
        current = frontier.get()

        if current[1].state == goalState:
            print("\nPath cost:", current[1].costPath)
            return current[1].goalPath

        for successor in getSuccessors(current[1]):
            s = hash(successor)
            if s not in reached or successor.costPath < reached[s].costPath:
                reached[s] = successor
                frontier.put((successor.costPath, successor))

    return "No solution found"


def aStar(initialNode, goalState):
    frontier = queue.PriorityQueue()
    reached = {}

    frontier.put((initialNode.costPath + initialNode.hCost, initialNode))
    reached[hash(initialNode)] = initialNode

    while not frontier.empty():
        current = frontier.get()

        if current[1].state == goalState:
            print("\nPath cost:", current[1].costPath)
            printState(current[1].state)
            return current[1].goalPath

        for successor in getSuccessors(current[1]):
            s = hash(successor)
            if s not in reached or successor.costPath < reached[s].costPath:
                reached[s] = successor
                frontier.put((successor.costPath + successor.hCost, successor))

    return "No solution found"

def solve(initialNode, goalState, format):
    if format == 0:
        return ucs(initialNode, goalState)
    elif format == 1:
        return aStar(initialNode, goalState)

# --------------------------------------------------------------------------------


def generate_unordered_puzzle(goalNode, moves):
    frontier = queue.PriorityQueue()
    reached = {}
    cnt = 0

    frontier.put((-(goalNode.costPath + goalNode.hCost), goalNode))
    reached[hash(goalNode)] = goalNode

    while not frontier.empty():
        current = frontier.get()

        if current[1].costPath == moves:
            # print("\nPath cost:", current[1].costPath)
            # printState(current[1].state)
            return current[1].state

        for successor in getSuccessors(current[1]):
            s = hash(successor)
            if s not in reached or successor.costPath > reached[s].costPath:
                reached[s] = successor
                frontier.put((-(successor.costPath + successor.hCost), successor))

    return "No solution found"

def getPuzzle():
    numberOfElement = int(input("Enter number of tiles (include 0 as blank space): "))
    print("Enter the puzzle state row by row, each row separated by spaces. Use '0' to represent the blank space.")
    puzzleState = []
    for i in range(numberOfElement):
        tile = int(input())
        puzzleState.append(tile)
    return puzzleState

def generateGoalState(n):
    if n == 3:
        return [1, 2, 3, 4, 5, 6, 7, 8, 0]
    if n == 4:
        return [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
    if n == 6:
        return [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 0]



def test1():
    # 6 steps:  [1,3,5,4,0,2,7,8,6]
    # 18 steps: [1,4,0,5,2,8,7,6,3]
    # 26 steps: [2,1,7,5,0,8,3,4,6]
    # 27 steps: [8,5,3,4,7,0,6,1,2]
    # 28 steps: [0,6,7,3,8,5,4,2,1]
    # 30 steps: [5,7,0,4,6,8,1,2,3]
    # 31 steps: [8,6,7,2,5,4,3,0,1]
    n = int(input("nxn puzzle - input n: "))
    goalState = generateGoalState(n)

    corGoal = getBlank(goalState)
    goalNode = Node(n, corGoal[0], corGoal[1], goalState, 0, 0, "")

    moves = int(input("Enter number of moves to unorder puzzle(may not accurate for small board and > 20 moves): "))

    initialState = generate_unordered_puzzle(goalNode, moves)

    cor = getBlank(initialState)
    initialNode = Node(n, cor[0], cor[1], initialState, 0, 0, "")

    inputAlgo = int(input("Input(0: UCS, 1:A*): "))
    if inputAlgo != 0 and inputAlgo != 1:
        return

    print("Init node")
    printState(initialNode.state)
    print()

    return initialNode, goalNode, inputAlgo

    # inputAlgo = int(input("Input(0: UCS, 1:A*): "))
    # if inputAlgo != 0 and inputAlgo != 1:
    #     return

    # print("Init node")
    # printState(initialNode.state)
    # print()
    # # %memit
    # %time
    # print(solve(initialNode, goalState, inputAlgo))


def test2():
    initialState = getPuzzle()
    acceptSize = [9, 16, 36]
    print(len(initialState))
    if len(initialState) not in acceptSize:
        print("illegal size")
        return
    n = int(math.sqrt(len(initialState)))
    goalState = generateGoalState(n)
    corGoal = getBlank(goalState)
    goalNode = Node(n, corGoal[0], corGoal[1], goalState, 0, 0, "")

    corInitial = getBlank(initialState)
    initialNode = Node(n, corInitial[0], corInitial[1], initialState, 0, 0, "")

    inputAlgo = int(input("Input(0: UCS, 1:A*): "))
    if inputAlgo != 0 and inputAlgo != 1:
        return

    print("Init node")
    printState(initialNode.state)
    print()

    return initialNode, goalNode, inputAlgo

    # print("Init node")
    # printState(initialNode.state)
    # print()
    # # %memit
    # %time
    # print(solve(initialNode, goalState, inputAlgo))


def test3():
    print("Test case for optimal moves = 18")
    n = int(input("Input n(nxn puzzle): "))

    goalState = generateGoalState(n)
    corGoal = getBlank(goalState)
    goalNode = Node(n, corGoal[0], corGoal[1], goalState, 0, 0, "")

    initialState = []
    if n == 3:
        initialState = [1,4,0,5,2,8,7,6,3]
    elif n == 4:
        initialState = [1,2,11,7,5,6,15,0,9,10,12,3,13,14,8,4]
    elif n == 6:
        initialState = [1,2,3,4,0,11,7,8,9,10,17,5,13,14,15,16,23,6,19,20,21,22,29,12,25,26,27,28,35,18,31,32,33,34,30,24]

    corInitial = getBlank(initialState)
    initialNode = Node(n, corInitial[0], corInitial[1], initialState, 0, 0, "")

    inputAlgo = int(input("Input(0: UCS, 1:A*): "))
    if inputAlgo != 0 and inputAlgo != 1:
        return

    print("Init node")
    printState(initialNode.state)
    print()
    return initialNode, goalNode, inputAlgo


def testChoice():
    print("This program doesn't have input error checking, please be careful.")
    c = int(input("0 to manually input a puzzle, 1 to shuffle a puzzle, 2 for built-in test cases: "))
    if c == 0:
        return test2()
    elif c == 1:
        return test1()
    elif c == 2:
        return test3()

res = testChoice()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
This program doesn't have input error checking, please be careful.
0 to manually input a puzzle, 1 to shuffle a puzzle, 2 for built-in test cases: 2
Test case for optimal moves = 18
Input n(nxn puzzle): 4
Input(0: UCS, 1:A*): 1
Init node
1 2 11 7 
5 6 15 0 
9 10 12 3 
13 14 8 4 



Measurement cell

In [38]:
%%time
%memit print(solve(res[0], res[1].state, res[2]))


Path cost: 18
1 2 3 4 
5 6 7 8 
9 10 11 12 
13 14 15 0 
3 4 8 12 15 11 7 3 4 8 12 15 11 7 3 4 8 12 
peak memory: 116.04 MiB, increment: 0.00 MiB
CPU times: user 91.2 ms, sys: 14.2 ms, total: 105 ms
Wall time: 209 ms
